# hw mac_id_ETL

In [1]:
d1 = spark.read.csv('/user/maxnet/database/sig.db/term_desc_all',sep='\x01')
d1 = d1.withColumnRenamed('_c0','term_id')\
       .withColumnRenamed('_c1','brand')\
       .withColumnRenamed('_c2','cn_name')\
       .withColumnRenamed('_c3','en_name')\
       .withColumnRenamed('_c4','type1')\
       .withColumnRenamed('_c5','type2')\
       .withColumnRenamed('_c6','os')\
       .withColumnRenamed('_c7','dtime')\
       .withColumnRenamed('_c8','price')\
       .withColumnRenamed('_c9','remarks')\
       .withColumnRenamed('_c10','prio')

d2 = spark.read.csv('/user/maxnet/database/sig.db/term_oui',sep='\x01')
d2 = d2.withColumnRenamed('_c0','mac_6').withColumnRenamed('_c1','manu')

from pyspark.sql.functions import *
d = spark.read.csv('/user/maxnet/database/sig.db/data_mac_res',sep='\x01')
d = d.withColumnRenamed('_c0','mac').withColumnRenamed('_c1','id')
d = d.withColumn('m1_6',substring(d.mac,1,6))

tmp_1 = d.join(d2,d.m1_6==d2.mac_6)

df = tmp_1.join(d1,tmp_1.id == d1.term_id).select('mac','id','manu','brand','type1','type2','cn_name').distinct().dropna()

hw = df.filter(df.brand == '华为')

hw = hw.withColumn('m1',substring(hw.mac,1,1))\
       .withColumn('m2',substring(hw.mac,2,1))\
       .withColumn('m3',substring(hw.mac,3,1))\
       .withColumn('m4',substring(hw.mac,4,1))\
       .withColumn('m5',substring(hw.mac,5,1))\
       .withColumn('m6',substring(hw.mac,6,1))\
       .withColumn('m7',substring(hw.mac,7,1))\
       .withColumn('m8',substring(hw.mac,8,1))\
       .withColumn('m9',substring(hw.mac,9,1))\
       .withColumn('m10',substring(hw.mac,10,1))\
       .withColumn('m11',substring(hw.mac,11,1))\
       .withColumn('m12',substring(hw.mac,12,1))  

tmp = hw.groupBy('cn_name').count().withColumnRenamed('count','c')
use = tmp.filter(tmp.c > 200000)
use_pandas = use.toPandas()

use_id = list(use_pandas.cn_name)

def filt(s):
    for x in use_id:
        if x == s:
            return 1
    else:
            return 0

ft = udf(lambda x:filt(x))
hw_tmp = hw.withColumn('filt',ft('cn_name'))
hw_final = hw_tmp.filter(hw_tmp.filt == '1').select('mac','id','cn_name','m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','m12')

hw_final = hw_final.withColumn('f1',conv(hw_final.m1, 16, 10)).withColumn('f2',conv(hw_final.m2, 16, 10))\
                         .withColumn('f3',conv(hw_final.m3, 16, 10)).withColumn('f4',conv(hw_final.m4, 16, 10))\
                         .withColumn('f5',conv(hw_final.m5, 16, 10)).withColumn('f6',conv(hw_final.m6, 16, 10))\
                         .withColumn('f7',conv(hw_final.m7, 16, 10)).withColumn('f8',conv(hw_final.m8, 16, 10))\
                         .withColumn('f9',conv(hw_final.m9, 16, 10)).withColumn('f10',conv(hw_final.m10, 16, 10))\
                         .withColumn('f11',conv(hw_final.m11, 16, 10)).withColumn('f12',conv(hw_final.m12, 16, 10))

hw_final = hw_final.select('mac','id','cn_name',\
                            col('f1').cast('float'),col('f2').cast('float'),\
                            col('f3').cast('float'),col('f4').cast('float'),\
                            col('f5').cast('float'),col('f6').cast('float'),\
                            col('f7').cast('float'),col('f8').cast('float'),\
                            col('f9').cast('float'),col('f10').cast('float'),\
                            col('f11').cast('float'),col('f12').cast('float'))

from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(hw_final)

from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

fractions = df_final.select('id').distinct().withColumn('fraction', lit(0.7)).rdd.collectAsMap()
trainDF = df_final.stat.sampleBy('id', fractions, seed=12)

train_mac = trainDF.select('mac').withColumnRenamed('mac','train_mac')
tmp = df_final.join(train_mac,df_final.mac==train_mac.train_mac,how='left')
testDF = tmp.filter(tmp.train_mac.isNull())

testDF = testDF.drop('train_mac')

trainDF.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/trainDF_hw',compression='gzip')
testDF.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/testDF_hw',compression='gzip')
#df_final.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/all_features',compression='gzip')